## The followinig code is used to plot the spectrogram that we get from the vggish model. 
There are a lot of problems with this model of code - 
1. we use the matplotlib to form all the spectrograms in the class that we have definied. Due to this all the matplotlib plots are presented as there is no way we can stop it from displaying all the spectrograms. So in later options we tried to solve this thing by using scipy.spectrogram to make the plots. Using this we dont have to see all the plots apart from the interactiive plot. 
2. We cannot store this notebook on github, so we use the dynamic map later instead of this notmal holoviews map. 

In [ ]:
# importing all the packages
import matplotlib.pyplot as plt

from scipy.io import wavfile
import io
import holoviews as hv
import param
from holoviews import opts
import xarray as xr

import os 
import numpy as np
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

import imp 
from holoviews import opts
from holoviews.operation import Operation

In [ ]:
fs, data = wavfile.read('soundfile.wav')

In [ ]:
# making the holoview once 
data_used = data.T
print(data_used.shape)
NFFT = 512  # the length of the windowing segments
plt.figure(figsize = (20,10))
#fig, ax1 = plt.subplots(nrows=1)
Pxx, freqs, bins, im = plt.specgram(data_used[0], NFFT=NFFT, Fs=fs, noverlap=10) # im is the picture of the plot

xr_spec = xr.DataArray(np.log(Pxx),dims = ('freq','time'),coords = {'freq':freqs,'time':bins})
xr_spec.name = 'Spectrogram'
hv_spec = hv.Dataset(xr_spec)

hv.extension('bokeh')
spec = hv_spec.to(hv.Image, ['time', 'freq'])

In [ ]:
from scipy import signal
freqs, bins, Pxx = signal.spectrogram(data_used[0], nperseg = 20, fs=fs, noverlap=10)

In [ ]:
plt.pcolormesh(bins, freqs, Pxx)
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.show()

In [ ]:
spec

In [ ]:
data_used = data.T 
print(data_used.shape)

In [ ]:
# Making a class of the above two functions to call when the holoviews is called 
data_used = data.T 

class interactive_plot(Operation):
    
    NFFT = param.Number(default=512)  # the length of the windowing segments
    
    def _process(self, element,  key=None):
        data_used = data.T    
        plt.figure(figsize = (20,10))
        #fig, ax1 = plt.subplots(nrows=1)
        Pxx, freqs, bins, im = plt.specgram(data_used[0], NFFT= self.p.NFFT, Fs=fs, noverlap=10) 
        xr_spec = xr.DataArray(np.log(Pxx),dims = ('freq','time'),coords = {'freq':freqs,'time':bins})
        xr_spec.name = 'Spectrogram'
        dataset = hv.Dataset(xr_spec)
        returning_thing = dataset.to(hv.Image, ['time', 'freq'])
        returning_thing
        
        return returning_thing

In [ ]:
hv.extension('bokeh', 'matplotlib')

hv.extension('bokeh')
output_notebook()

imp.reload(hv)

widget = hv.HoloMap({NFFT: interactive_plot(spec, NFFT=NFFT) 
                      for NFFT in range(16, 512, 8)}, kdims="NFFT")

widget.opts(framewise=True)